In [2]:
#import whatever you need here
import pandas as pd
import sklearn
import ast
import math
import pickle
import os
import glob
from numpy import * 
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB,ComplementNB

# Load data

In [ ]:
#load the training data here, and process however you like

path = "/Users/wangqiaowen/Downloads/PAI2020_assignment2/example_player/training_data"                    
all_files = glob.glob(os.path.join(path, "*.csv"))     

training_data = []

#load the data into a pandas frames
for file in all_files:
    game_data = pd.read_csv(file,index_col=False)
    reason = game_data.iloc[-1][6]
    
    #if the reason is found, add it to each line to fill out the blanks
    if type(reason) is str:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = reason

    #else there was no reason, implying the game reached the number of iterations
    else:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = 'max_iterations'    
    
    training_data.append(game_data)

# Feature selection and Instance selection 

## Sheep


- feature 1:determine if wolf within two steps up
- feature 2:determine if wolf within two steps down
- feature 3:determine if wolf within two steps left
- feature 4:determine if wolf within two steps right
- feature 5:determine if closest food within two steps up
- feature 6:determine if closest food is above the sheep
- feature 7:determine if closest food is right of the sheep
- feature 8:determine if closest food is left of the sheep
- feature 9:manhattan distance between sheep and wolf

The data I needed were the positions of food, the positions of wolves and sheep

In [ ]:

X_sheep = []
Y_sheep = []
number_moves = 0

count = 0
for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:  #player 2 wins
        sheep = 's'
        wolf = 'W'
        op_wolf = 'w'
        op_sheep = 'S'
        
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:  #player 1 wins
        sheep = 'S'
        wolf = 'w'
        op_wolf = 'W'
        op_sheep = 's'
    else:
        continue

    rhubarb = 'r'
    grass = 'g'
    fence = '#'

    #for each game state in our training data
    for index,row in game.iterrows():
        
         #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] == 'player1 wolf' or row['turn_made_by'] == 'player2 wolf':
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field= ast.literal_eval(row['field_before'])
        
        #get positions of sheep, wolf and food items
        food = []
        obstacles = []
        y=0
        for field_row in field:
            x = 0
            for item in field_row:
                if item == sheep:
                    sheep_position = (x,y)
                elif item == wolf:
                    wolf_position = (x,y)
                elif item == rhubarb or item == grass:
                    food.append((x,y))
                elif item == fence:
                    obstacles.append((x,y))
                elif item == op_wolf:
                    op_wolf_position = (x,y)
                elif item == op_sheep:
                    op_sheep_position = (x,y)
                x += 1
            y+=1
        
        #feature 1: determine if wolf within two steps up
        if sheep_position[1] - wolf_position[1] <= 2 and sheep_position[1] - wolf_position[1] > 0:
            s_feature1 = 1
        else:
            s_feature1 = 0
        game_features.append(s_feature1)

        #feature 2: determine if wolf within two steps down
        if sheep_position[1] - wolf_position[1] >= -2 and sheep_position[1] - wolf_position[1] < 0:
            s_feature2 = 1
        else:
            s_feature2 = 0
        game_features.append(s_feature2)

        #feature 3: determine if wolf within two steps left
        if sheep_position[0] - wolf_position[0] <= 2 and sheep_position[0] - wolf_position[0] > 0:
            s_feature3 = 1
        else:
            s_feature3 = 0
        game_features.append(s_feature3)

        #feature 4: determine if wolf within two steps right
        if sheep_position[0] - wolf_position[0] >= -2 and sheep_position[0] - wolf_position[0] < 0:
            s_feature4 = 1
        else:
            s_feature4 = 0
        game_features.append(s_feature4)
        
        s_feature5 = 0
        s_feature6 = 0
        s_feature7 = 0
        s_feature8 = 0

        #determine closest food:
        food_distance = 1000
        food_goal = None
        for food_item in food:
            distance = abs(food_item[0] - sheep_position[0]) + abs(food_item[1] - sheep_position[1])
            if distance < food_distance:
                food_distance = distance
                food_goal = food_item
#             elif distance == food_distance and field[food_item[1]][food_item[0]] == rhubarb :
                
        
        if food_goal != None:
            #feature 5: determine if closest food is below the sheep
            if sheep_position[1] - food_goal[1] < 0:
                s_feature5 = 1

            #feature 6: determine if closest food is above the sheep
            if sheep_position[1] - food_goal[1] > 0:
                s_feature6 = 1

            #feature 7: determine if closest food is right of the sheep
            if sheep_position[0] - food_goal[0] < 0:
                s_feature7 = 1

            #feature 8: determine if closest food is left of the sheep
            if sheep_position[0] - food_goal[0] > 0:
                s_feature8 = 1

        game_features.append(s_feature5)
        game_features.append(s_feature6)
        game_features.append(s_feature7)
        game_features.append(s_feature8)
        #feature 9:manhattan distance between sheep and wolf
        s_feature9 = abs(sheep_position[0]-wolf_position[0])+abs(sheep_position[1]-wolf_position[1])
        
        game_features.append(s_feature9)
        
        X_sheep.append(game_features)
        Y_sheep.append(move)



## Wolf


- feature 1:determine if the sheep is above the wolf
- feature 2:determine if the sheep is below the wolf
- feature 3:determine if the sheep is left of the wolf
- feature 4:determine if the sheep is right from the wolf
- feature 5:manhattan distance between sheep and wolf
- feature 6:manhattan distance between two wolves

The data I needed were the positions of wolves and sheep


In [ ]:

X_wolf = []
Y_wolf = []
number_moves = 0

count =0 
for game in training_data:

    #for each game state in our training data
    for index,row in game.iterrows():
        
        #we only want games where the sheep was eaten (=the wolf was successful)
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten'):
            continue
        
        #we want to only learn from wolves
        if row['turn_made_by'] == 'player1 sheep' or row['turn_made_by'] == 'player2 sheep':
            continue
        
        number_moves += 1
        
        
        #we want to learn from the wolf that ate the sheep
        if row['reason'] == 'sheep1 eaten':
            sheep = 'S'
            wolf = 'w'
            op_wolf = 'W'
            my_sheep = 's'
        else:
            sheep = 's'
            wolf = 'W'
            op_wolf = 'w'
            my_sheep = 'S'
            
        rhubarb = 'r'
        grass = 'g'

        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field= ast.literal_eval(row['field_before'])
        
        #get positions of sheep, wolf and food items
        y=0
        obstacles = []
        for field_row in field:
            x = 0
            for item in field_row:
                if item == sheep:
                    sheep_position = (x,y)
                elif item == wolf:
                    wolf_position = (x,y)
                elif item == op_wolf:
                    op_wolf_position = (x,y)
                elif item == my_sheep:
                    my_sheep_position = (x,y)
                x += 1
            y+=1
            
        #feature 1: determine if the sheep is above the wolf
        if wolf_position[1] - sheep_position[1] > 0:
            w_feature1 = 1
        else:
            w_feature1 = 0
        game_features.append(w_feature1)

        #feature 2: determine if the sheep is below the wolf
        if wolf_position[1] - sheep_position[1] < 0:
            w_feature2 = 1
        else:
            w_feature2 = 0
        game_features.append(w_feature2)

        #feature 3: determine if the sheep is left of the wolf
        if wolf_position[0] - sheep_position[0] > 0:
            w_feature3 = 1
        else:
            w_feature3 = 0
        game_features.append(w_feature3)

        #feature 4: determine if the sheep is right from the wolf
        if wolf_position[0] - sheep_position[0] < 0:
            w_feature4 = 1
        else:
            w_feature4 = 0
        game_features.append(w_feature4)

        #feature 5:manhattan distance between sheep and wolf
        s_feature5 = abs(sheep_position[0]-wolf_position[0])+abs(sheep_position[1]-wolf_position[1])
        
        game_features.append(s_feature5)
        
        #feature 6:manhattan distance between two wolves
        s_feature6 = abs(op_wolf_position[0]-wolf_position[0])+abs(op_wolf_position[1]-wolf_position[1])
        
        game_features.append(s_feature6)
        X_wolf.append(game_features)
        Y_wolf.append(move)
        

# Train sheep

In [22]:
x_train,x_test,y_train,y_test = train_test_split(X_sheep,Y_sheep,test_size=0.3,random_state=0)

In [8]:
sheep_GNB = GaussianNB()
sheep_GNB.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

# Train wolf

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_wolf,Y_wolf,test_size=0.3,random_state=0)

In [23]:
wolf_GNB = GaussianNB()
wolf_GNB.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

# Save models to files

Save your models to files here using pickle. Change the [uzhshortname] to your own UZH shortname. This name needs to match the model that you caller in your python player file.

In [23]:
sheep_filename = 'qiawan_sheep_model.sav'
wolf_filename = 'qiawan_wolf_model.sav'

pickle.dump(sheep_GNB,open(sheep_filename,'wb'))
pickle.dump(wolf_GNB,open(wolf_filename,'wb'))